In [ ]:
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

# DATA PREPARATION

In [ ]:
dataset_path = "/content/dataset/rps-cv-images"

CONFIG = {
    "IM_SIZE" : 256,
    "BATCH_SIZE" : 32,
}

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(CONFIG["IM_SIZE"],CONFIG["IM_SIZE"]),
    batch_size=CONFIG["BATCH_SIZE"],
    subset="training",
    validation_split=0.2,
    shuffle = True,
    seed = 123
)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=(CONFIG["IM_SIZE"],CONFIG["IM_SIZE"]),
    batch_size=CONFIG["BATCH_SIZE"],
    subset="validation",
    validation_split=0.2,
    shuffle = True,
    seed = 123
)

In [ ]:
class_names = train_ds.class_names
print(class_names)

# DATA VISUALIZE

In [ ]:
plt.figure(figsize = (12,12))

for images,labels in train_ds.take(1):
  for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow(images[i]/255)
    plt.title(class_names[labels[i].numpy()])
    plt.axis("off")

# DATA AUGMENTATION

In [ ]:
data_augmentation_layer = tf.keras.models.Sequential([
    tf.keras.layers.RandomFlip("horizontal", input_shape=(CONFIG["IM_SIZE"],CONFIG["IM_SIZE"],3)),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
])

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation_layer(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

In [ ]:
train_ds = train_ds.shuffle(1000).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
print(f"Train Size: {len(train_ds)}, Val size: {len(val_ds)}")
print(train_ds)
print(val_ds)

# TRANSFER LEARNING WITH VGG

In [ ]:
vgg_backbone = tf.keras.applications.vgg16.VGG16(
    include_top = False,
    weights = 'imagenet',
    input_shape = (CONFIG["IM_SIZE"],CONFIG["IM_SIZE"],3)
)
vgg_backbone.trainable = False

In [ ]:
vgg_model = tf.keras.models.Sequential([
    vgg_backbone,
    layers.Flatten(),
    layers.Dense(1000,activation = "relu"),
    layers.Dropout(0.2),
    layers.Dense(len(class_names),activation="softmax")
])
vgg_model.summary()

In [ ]:
vgg_model.compile(optimizer="sgd",loss = tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
history = vgg_model.fit(train_ds,validation_data=val_ds,epochs = 15)

In [ ]:
vgg_model.evaluate(val_ds)

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("VGG loss")
plt.legend(["loss","val_loss"])
plt.show()

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("VGG accuracy")
plt.legend(["accuracy","val_accuracy"])
plt.show()

# SEQUENTIAL MODEL

In [ ]:
model = tf.keras.models.Sequential([
    layers.Rescaling(1./255,input_shape = (CONFIG["IM_SIZE"],CONFIG["IM_SIZE"],3)),

    data_augmentation_layer,

    layers.Conv2D(filters = 32,kernel_size=3,strides = 1,padding = "valid",activation="relu"),
    layers.MaxPool2D(),

    layers.Conv2D(filters = 64,kernel_size=3,strides = 1,padding = "valid",activation="relu"),
    layers.MaxPool2D(),

    layers.Conv2D(filters = 128,kernel_size=3,strides = 1,padding = "valid",activation="relu"),
    layers.MaxPool2D(),

    layers.Dropout(0.3),


    layers.Flatten(),

    layers.Dense(128,activation="relu"),
    layers.Dense(100,activation="relu"),
    layers.Dense(len(class_names),activation="softmax"),

])
model.summary()

In [ ]:
model.compile(optimizer="adam",loss = tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds,validation_data=val_ds,epochs=10,batch_size= CONFIG["BATCH_SIZE"])

In [ ]:
model.evaluate(val_ds)

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title('Model loss')
plt.legend(["train_loss","val_loss"])
plt.show()

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title('Model acc')
plt.legend(["train_loss","val_loss"])
plt.show()

# PREDICT DATA

In [ ]:
plt.figure(figsize=(16,16))

for images,labels in val_ds.take(1):
  for i in range(25):
    plt.subplot(5,5,i+1)
    plt.imshow(images[i]/255)
    true_label = class_names[labels[i].numpy()]
    img_arr = tf.expand_dims(images[i],axis = 0)
    prediction = np.argmax(vgg_model.predict(img_arr))
    plt.title(f"True: {true_label} \n Predict: {class_names[prediction]}")
    plt.axis("off")